In [3]:
import warnings

# Suppress specific deprecation warnings from protobuf
warnings.filterwarnings("ignore", category=UserWarning, module='google.protobuf')

In [4]:
import os
import cv2
import numpy as np
import pickle
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.framework.formats import landmark_pb2

# Initialize MediaPipe Face Landmarker
base_options = python.BaseOptions(model_asset_path='face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)

# Load the pre-trained model (for inference)
with open('FaceShapeModel_RandomForest.pkl', 'rb') as f:
    face_shape_model = pickle.load(f)

# Define function to compute Euclidean distance in 3D
def distance_3d(p1, p2):
    return np.sqrt(np.sum((np.array(p1) - np.array(p2)) ** 2))


def calculate_face_features(landmarks):
    landmark_indices = {
        'forehead': 10,
        'chin': 152,
        'left_cheek': 234,
        'right_cheek': 454,
        'left_eye': 263,
        'right_eye': 33
    }
    features = []
    landmarks_dict = {name: landmarks[idx] for name, idx in landmark_indices.items()}
    features.append(distance_3d(landmarks_dict['forehead'], landmarks_dict['chin']))
    features.append(distance_3d(landmarks_dict['left_cheek'], landmarks_dict['right_cheek']))
    features.append(distance_3d(landmarks_dict['left_eye'], landmarks_dict['right_eye']))
    return np.array(features)

def get_face_shape_label(label):
    shapes = ["Heart", "Oval", "Round", "Square"]
    return shapes[label]

def draw_landmarks_on_image(rgb_image, detection_result):
    face_landmarks_list = detection_result.face_landmarks
    annotated_image = np.copy(rgb_image)
    if face_landmarks_list:
        for face_landmarks in face_landmarks_list:
            face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
            face_landmarks_proto.landmark.extend([
                landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
            ])
            mp.solutions.drawing_utils.draw_landmarks(
                image=annotated_image,
                landmark_list=face_landmarks_proto,
                connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp.solutions.drawing_styles.get_default_face_mesh_tesselation_style())
            mp.solutions.drawing_utils.draw_landmarks(
                image=annotated_image,
                landmark_list=face_landmarks_proto,
                connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp.solutions.drawing_styles.get_default_face_mesh_contours_style())
            mp.solutions.drawing_utils.draw_landmarks(
                image=annotated_image,
                landmark_list=face_landmarks_proto,
                connections=mp.solutions.face_mesh.FACEMESH_IRISES,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp.solutions.drawing_styles.get_default_face_mesh_iris_connections_style())
    return annotated_image

def process_images_from_folder(folder_path, expected_label):
    total_images = 0
    correct_predictions = 0

    for img_file in os.listdir(folder_path):
        img_pth = os.path.join(folder_path, img_file)
        img = cv2.imread(img_pth)

        if img is None:
            print(f"Error loading image {img_pth}. Skipping...")
            continue

        rgb_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_image)
        detection_result = detector.detect(image)

        if detection_result.face_landmarks:
            for face_landmarks in detection_result.face_landmarks:
                landmarks = [[lm.x, lm.y, lm.z] for lm in face_landmarks]
                landmarks = np.array(landmarks)
                face_features = calculate_face_features(landmarks)
                face_shape_label = face_shape_model.predict([face_features])[0]
                predicted_shape = get_face_shape_label(face_shape_label)

                if predicted_shape == expected_label:
                    correct_predictions += 1

                total_images += 1
        else:
            print(f"No face landmarks detected for image {img_file}. Skipping...")

    accuracy = correct_predictions / total_images if total_images > 0 else 0
    return accuracy

# Path to the main folder
face_shape_folder = "Face_Shape"

# Separate folders for training and testing data
training_folder = os.path.join(face_shape_folder, "training")
testing_folder = os.path.join(face_shape_folder, "testing")

# Subfolder names for face shapes
subfolders = ["Heart", "Oval", "Round", "Square"]

def evaluate_model_on_dataset(dataset_folder, dataset_name):
    print(f"\nEvaluating model on {dataset_name} dataset:\n")
    for subfolder in subfolders:
        folder_path = os.path.join(dataset_folder, subfolder)
        if os.path.isdir(folder_path):
            accuracy = process_images_from_folder(folder_path, subfolder)
            print(f"Accuracy for {subfolder} images: {accuracy:.2%}")
        else:
            print(f"Folder {folder_path} does not exist.")

# Evaluate the model on training dataset
evaluate_model_on_dataset(training_folder, "Training")

# Evaluate the model on testing dataset
evaluate_model_on_dataset(testing_folder, "Testing")



Evaluating model on Training dataset:

Accuracy for Heart images: 0.00%
Accuracy for Oval images: 2.08%
Accuracy for Round images: 0.83%
Accuracy for Square images: 99.17%

Evaluating model on Testing dataset:

Accuracy for Heart images: 0.00%
Accuracy for Oval images: 2.08%
Accuracy for Round images: 0.00%
Accuracy for Square images: 100.00%
